In [ ]:
'''Converting BioRails Templates & Aurigene Files'''
import asyncio
import os
import openpyxl
import pandas as pd
from moleculedb.models.moleculedb import ApprovedMolecule
from moleculedb.models.virtualdb import VirtualMolecule
from moleculedb.utils.database import init

async def convert_excel_file(file_path: str):
    await init(os.environ['db_name'], os.environ['db_connect_string'])
    df = openpyxl.load_workbook(file_path)
    wb = df.active
    description = f"{wb['A1'].value[:-6]} - {wb['B4'].value}"
    date = f"{wb['B3'].value}".split(" ")[0]
    team = '1910 Genetics'
    user = 'System Administrator'
    state = 'Unpublished'
    assay = description
    site = 'Aurigene'
    eln = f"{wb['B6'].value}"
    compounds = []
    for cell in wb['A']:
        if cell.value:
            mol = None
            if "1910-" in cell.value:
                mol = await ApprovedMolecule.find({
                    "molecule_id": cell.value[5:]
                }).to_list()
                if not mol:
                    mol = await ApprovedMolecule.find({
                        "parent_id": int(cell.value[5:-3])
                    }).to_list()
            if "1910V" in cell.value:
                mol = await ApprovedMolecule.find({
                    "alias": cell.value
                }).to_list()
            if mol:
                mol_id = mol[0].molecule_id
                project = mol[0].target[0]
                if project == "NTG002-1":
                    project = "NTG002"
                target = mol[0].real_target
                compounds.append({
                    "name": f"{user}-{assay} at {site} for {project} on {date}",
                    "description": description,
                    "state": state,
                    "team": team,
                    "user": user,
                    "#label": "",
                    "batch.#method": "ChromlogD",
                    "batch.#valid": 1,
                    "batch:batch": mol_id,
                    "batch:compound_name": f'1910-{mol_id}',
                    "batch:project_alias": project,
                    "batch:conc": 100,
                    "batch:logD": wb[f'B{cell.coordinate[1:]}'].value,
                    "batch:pH": 2.6,
                    "batch:target": target,
                    "batch:comments": 0,
                    "batch:assay_site": site,
                    "batch:eln_reference": eln,
                    "batch:execution_date": date,
                })
                compounds.append({
                    "name": f"{user}-{assay} at {site} for {project} on {date}",
                    "description": description,
                    "state": state,
                    "team": team,
                    "user": user,
                    "#label": "",
                    "batch.#method": "ChromlogD",
                    "batch.#valid": 1,
                    "batch:batch": mol_id,
                    "batch:compound_name": f'1910-{mol_id}',
                    "batch:project_alias": project,
                    "batch:conc": 100,
                    "batch:logD": wb[f'C{cell.coordinate[1:]}'].value,
                    "batch:pH": 7.4,
                    "batch:target": target,
                    "batch:comments": 0,
                    "batch:assay_site": site,
                    "batch:eln_reference": eln,
                    "batch:execution_date": date,
                })
                compounds.append({
                    "name": f"{user}-{assay} at {site} for {project} on {date}",
                    "description": description,
                    "state": state,
                    "team": team,
                    "user": user,
                    "#label": "",
                    "batch.#method": "ChromlogD",
                    "batch.#valid": 1,
                    "batch:batch": mol_id,
                    "batch:compound_name": f'1910-{mol_id}',
                    "batch:project_alias": project,
                    "batch:conc": 100,
                    "batch:logD": wb[f'D{cell.coordinate[1:]}'].value,
                    "batch:pH": 10.5,
                    "batch:target": target,
                    "batch:comments": 0,
                    "batch:assay_site": site,
                    "batch:eln_reference": eln,
                    "batch:execution_date": date,
                })
    df = pd.DataFrame.from_records(compounds)
    for idx, row in df.iterrows():
        if not type(df.at[idx, 'batch:logD']) == float:
            df.at[idx, 'batch:comments'] = df.at[idx, 'batch:logD']
            df.at[idx, 'batch:logD'] = -999
            df.at[idx, 'batch.#valid'] = 0
    #create some data with Names column
    #create unique list of names
    UniqueNames = df['batch:project_alias'].unique()
    # #create a data frame dictionary to store your data frames
    DataFrameDict = {elem: pd.DataFrame() for elem in UniqueNames}
    for key in DataFrameDict.keys():
        DataFrameDict[key] = df[:][df['batch:project_alias'] == key]
        DataFrameDict[key].to_excel(
            f"{DataFrameDict[key]['name'].iloc[0]}.xlsx",
            index=False,
            sheet_name='DataLoad')
files = [
    # 'Curated data campaign 1383.xlsx',
    # 'Curated data Campaign 1458.xlsx',
    # 'Curated data Campaign 1487.xlsx',
    # 'Curated data campaign 1396.xlsx',
    # 'Curated data Campaign 1499.xlsx',
    # 'Curated data campaign 1510.xlsx',
    # 'Curated data Campaign 1598.xlsx',
    # 'curated data campaign 1441.xlsx',
]
if __name__ == "__main__":
    for file in files:
        asyncio.run(
            convert_excel_file(f'/Users/admin/Downloads/Curated data/{file}'))